In [5]:
from plotly.subplots import make_subplots
from plotting import plot_values, update_layout
import glob
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import json
import matplotlib.pyplot as plt

### Get the names of all files we want to read 

In [6]:
records = []
for experiment_file in glob.glob("../data/*/*/params.json", recursive=True):
    with open(experiment_file, "r", encoding="utf-8") as file:
        records.append(json.load(file))
data = pd.DataFrame.from_records(records)
data.head()

,delta_std,deltas_used,dir_path,env_name,filter,n_directions,n_iter,n_workers,policy_type,rollout_length,seed,shift,step_size
0,0.025,4,data/SafetyHopperVelocity-v1/1724913801.39389,SafetyHopperVelocity-v1,MeanStdFilter,8,1000,10,linear,1000,0,1,0.01
1,0.025,4,data/SafetyHopperVelocity-v1/1724914819.3440247,SafetyHopperVelocity-v1,MeanStdFilter,8,1000,10,linear,1000,1,1,0.01
2,0.025,4,data/SafetyHopperVelocity-v1/1724917030.6341782,SafetyHopperVelocity-v1,MeanStdFilter,8,1000,10,linear,1000,3,1,0.01
3,0.025,4,data/SafetyHopperVelocity-v1/1724915895.0363967,SafetyHopperVelocity-v1,MeanStdFilter,8,1000,10,linear,1000,2,1,0.01
4,0.030,4,data/SafetyHalfCheetahVelocity-v1/1724913852.8...,SafetyHalfCheetahVelocity-v1,MeanStdFilter,32,1000,10,linear,1000,0,0,0.02


In [7]:
def plot_env(env_name, data):
    df = []
    for filename, seed in zip(data.dir_path, data.seed):
        x = pd.read_csv("../"+filename+"/log.txt", sep="\t")
        df.append(x)
    df = pd.concat(df, axis=0).groupby("Iteration")[["timesteps", "AverageReward"]].mean().reset_index()
    df.drop("Iteration", axis=1, inplace=True)
    df.rename({
        'timesteps': "steps",
        'AverageReward': "reward"
    }, axis=1, inplace=True)
    df.steps = df.steps.astype(int)
    df["env_name"] = env_name
    fig = px.line(data_frame=df, x="steps", y="reward")
    update_layout(fig, env_name.replace("Safety","").replace("Velocity", ""), "Total Env Interacts", "Reward", row=1, col=1, upkwargs=dict(width=500, height=400))
    fig.write_image(f"../images/{env_name}.png", scale=3)
    return df

In [8]:
for env_name, df in data.groupby("env_name"):
    plot_env(env_name, df)
